<a href="https://colab.research.google.com/github/hwangho-kim/Utility-OAC/blob/main/CSV_%ED%8C%8C%EC%9D%BC_%EC%B9%BC%EB%9F%BC_%ED%8F%89%EA%B7%A0_%EA%B3%84%EC%82%B0_%EB%B0%8F_%EC%A0%80%EC%9E%A5_%EC%8A%A4%ED%81%AC%EB%A6%BD%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os
import re
from collections import defaultdict

def process_csv_files_with_averaging(input_folder, output_folder):
    """
    지정된 폴더의 CSV 파일들을 읽어 특정 패턴의 칼럼들의 평균을 계산하고,
    결과를 새로운 CSV 파일로 저장합니다.

    Args:
        input_folder (str): 입력 CSV 파일들이 있는 폴더 경로.
        output_folder (str): 처리된 CSV 파일들을 저장할 폴더 경로.
    """
    # 출력 폴더가 존재하지 않으면 생성합니다.
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        print(f"출력 폴더 '{output_folder}'를 생성했습니다.")

    # 평균 계산 대상 칼럼을 식별하기 위한 정규 표현식입니다.
    # 패턴: basename_X 또는 basename_X_Y (여기서 X는 1, 2, 또는 3이고, Y는 숫자입니다)
    # 예: data_1, feature_2_10, metric_3_5
    column_pattern = re.compile(r"^(.*?)_([1-3])(_\d+)?$")

    # 입력 폴더 내의 모든 파일을 확인합니다.
    for filename in os.listdir(input_folder):
        if filename.endswith(".csv"):
            input_file_path = os.path.join(input_folder, filename)
            # 출력 파일 이름은 원본 파일 이름에 "_AVG"를 추가합니다.
            output_file_name = f"{os.path.splitext(filename)[0]}_AVG.csv"
            output_file_path = os.path.join(output_folder, output_file_name)

            print(f"파일 '{filename}' 처리 중...")

            try:
                # CSV 파일을 읽어들입니다.
                df = pd.read_csv(input_file_path)

                if df.empty:
                    print(f"  파일 '{filename}'이 비어있습니다. 원본 내용 그대로 '{output_file_name}'으로 저장합니다.")
                    df.to_csv(output_file_path, index=False, encoding='utf-8-sig')
                    continue

                # 평균을 계산할 칼럼들을 그룹별로 저장할 딕셔너리입니다.
                cols_to_average_map = defaultdict(list)
                # 평균 계산에 사용되지 않고 그대로 유지될 칼럼들의 리스트입니다.
                original_cols_to_keep = []

                # 데이터프레임의 모든 칼럼을 순회합니다.
                for col in df.columns:
                    match = column_pattern.match(col)
                    if match:
                        # 정규 표현식에 매칭되면, 베이스 이름(접두사)을 추출합니다.
                        base_name = match.group(1)
                        cols_to_average_map[base_name].append(col)
                    else:
                        # 매칭되지 않으면 원본 유지 칼럼 리스트에 추가합니다.
                        original_cols_to_keep.append(col)

                # 최종 결과 데이터프레임을 구성할 파트들을 담을 리스트입니다.
                # 먼저, 평균 계산에 사용되지 않는 원본 칼럼들을 추가합니다.
                result_df_parts = [df[original_cols_to_keep].copy()] # .copy() to avoid SettingWithCopyWarning

                # 그룹화된 칼럼들의 평균을 계산하고 결과 파트에 추가합니다.
                if not cols_to_average_map:
                    print(f"  파일 '{filename}'에서 평균 계산 대상 칼럼을 찾지 못했습니다.")

                for base_name, related_cols in cols_to_average_map.items():
                    if related_cols:
                        # 평균 계산 전에 해당 칼럼들을 숫자형으로 변환 시도합니다.
                        # 숫자로 변환할 수 없는 값은 NaN으로 처리됩니다.
                        numeric_cols_df = df[related_cols].apply(pd.to_numeric, errors='coerce')

                        avg_col_name = f"{base_name}_AVG"
                        # row별 평균을 계산합니다 (NaN 값은 무시).
                        # 해당 row의 모든 값이 NaN이면 결과도 NaN이 됩니다.
                        avg_series = numeric_cols_df.mean(axis=1)
                        avg_series.name = avg_col_name
                        result_df_parts.append(avg_series)
                        print(f"  칼럼 그룹 '{base_name}' (칼럼: {related_cols})의 평균을 '{avg_col_name}'으로 계산했습니다.")

                # 모든 파트를 하나로 합쳐 최종 데이터프레임을 만듭니다.
                if not result_df_parts:
                    # 이 경우는 df가 비어있지 않다면 발생하기 어렵습니다.
                    final_df = pd.DataFrame()
                else:
                    final_df = pd.concat(result_df_parts, axis=1)

                # 처리된 데이터프레임을 CSV 파일로 저장합니다.
                # 한글 깨짐 방지를 위해 'utf-8-sig' 인코딩을 사용합니다.
                final_df.to_csv(output_file_path, index=False, encoding='utf-8-sig')
                print(f"  파일 '{filename}' 처리 완료. 저장 위치: '{output_file_path}'")

            except pd.errors.EmptyDataError:
                print(f"  파일 '{filename}'이 비어있거나 유효한 CSV 형식이 아닙니다. 건너<0xEB><0><0xB5>니다.")
            except Exception as e:
                print(f"  파일 '{filename}' 처리 중 오류 발생: {e}")
    print("\n모든 CSV 파일 처리가 완료되었습니다.")

if __name__ == '__main__':
    # 사용 예시:
    # 아래 'your_input_csv_folder_path'와 'your_output_csv_folder_path'를
    # 실제 CSV 파일들이 있는 폴더 경로와 결과를 저장할 폴더 경로로 변경해주세요.

    # 예: input_directory = r"C:\Users\YourUser\Desktop\InputCSVs"
    # 예: output_directory = r"C:\Users\YourUser\Desktop\OutputCSVs"

    input_directory = "your_input_csv_folder_path"  # << 여기에 입력 CSV 폴더 경로를 입력하세요.
    output_directory = "your_output_csv_folder_path" # << 여기에 출력 CSV 폴더 경로를 입력하세요.

    if input_directory == "your_input_csv_folder_path" or \
       output_directory == "your_output_csv_folder_path":
        print("스크립트 하단의 'input_directory'와 'output_directory' 변수에\n"
              "실제 폴더 경로를 입력한 후 다시 실행해주세요.")
    else:
        process_csv_files_with_averaging(input_directory, output_directory)